In [2]:
from itertools import product

def valid_pattern(pattern):
    # Count occurrences of 'H' and 'A'
    h_count = pattern.count('H')
    a_count = pattern.count('A')

    # Check for similar number of H and A matches (4H-5A or 5H-4A)
    if not (h_count == 4 and a_count == 5 or h_count == 5 and a_count == 4):
        return False

    # Check for more than two consecutive H or A matches
    for i in range(len(pattern) - 2):
        if pattern[i] == pattern[i + 1] == pattern[i + 2]:
            return False

    return True

# Generate all combinations of H and A for 9 matches
all_patterns = product('HA', repeat=9)

# Filter patterns that meet the criteria
valid_patterns = [''.join(pattern) for pattern in all_patterns if valid_pattern(pattern)]

# Display the valid patterns
for pattern in valid_patterns:
    print(pattern)

# Output the number of valid patterns
print(f"Total valid patterns: {len(valid_patterns)}")



HHAHHAHAA
HHAHHAAHA
HHAHAHHAA
HHAHAHAHA
HHAHAHAAH
HHAHAAHHA
HHAHAAHAH
HHAHAAHAA
HHAAHHAHA
HHAAHHAAH
HHAAHAHHA
HHAAHAHAH
HHAAHAHAA
HHAAHAAHH
HHAAHAAHA
HAHHAHHAA
HAHHAHAHA
HAHHAHAAH
HAHHAAHHA
HAHHAAHAH
HAHHAAHAA
HAHAHHAHA
HAHAHHAAH
HAHAHAHHA
HAHAHAHAH
HAHAHAHAA
HAHAHAAHH
HAHAHAAHA
HAHAAHHAH
HAHAAHHAA
HAHAAHAHH
HAHAAHAHA
HAHAAHAAH
HAAHHAHHA
HAAHHAHAH
HAAHHAHAA
HAAHHAAHH
HAAHHAAHA
HAAHAHHAH
HAAHAHHAA
HAAHAHAHH
HAAHAHAHA
HAAHAHAAH
HAAHAAHHA
HAAHAAHAH
AHHAHHAHA
AHHAHHAAH
AHHAHAHHA
AHHAHAHAH
AHHAHAHAA
AHHAHAAHH
AHHAHAAHA
AHHAAHHAH
AHHAAHHAA
AHHAAHAHH
AHHAAHAHA
AHHAAHAAH
AHAHHAHHA
AHAHHAHAH
AHAHHAHAA
AHAHHAAHH
AHAHHAAHA
AHAHAHHAH
AHAHAHHAA
AHAHAHAHH
AHAHAHAHA
AHAHAHAAH
AHAHAAHHA
AHAHAAHAH
AHAAHHAHH
AHAAHHAHA
AHAAHHAAH
AHAAHAHHA
AHAAHAHAH
AHAAHAAHH
AAHHAHHAH
AAHHAHHAA
AAHHAHAHH
AAHHAHAHA
AAHHAHAAH
AAHHAAHHA
AAHHAAHAH
AAHAHHAHH
AAHAHHAHA
AAHAHHAAH
AAHAHAHHA
AAHAHAHAH
AAHAHAAHH
AAHAAHHAH
AAHAAHAHH
Total valid patterns: 90


In [3]:
# Dictionary `feasible_opponents` where feasible_opponents[i, t]
# is the set of feasible opponents for pattern i in week t


# Initialize the feasible opponents dictionary
feasible_opponents = {}

# Iterate through each pattern and week
num_patterns = len(valid_patterns)
num_weeks = len(valid_patterns[0])

for i in range(num_patterns):
    for t in range(num_weeks):
        # Get the H/A status of pattern i in week t
        current_status = valid_patterns[i][t]
        
        # Find all patterns that have the opposite status in week t
        feasible_opponents[i, t] = {j for j in range(num_patterns) if j != i and valid_patterns[j][t] != current_status}


# Print the feasible opponents dictionary for verification
for key, value in feasible_opponents.items():
    print(f"Pattern {key[0]} in week {key[1]} can play against patterns {value}")


Pattern 0 in week 0 can play against patterns {45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89}
Pattern 0 in week 1 can play against patterns {15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89}
Pattern 0 in week 2 can play against patterns {15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89}
Pattern 0 in week 3 can play against patterns {8, 9, 10, 11, 12, 13, 14, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 69, 70, 71, 72, 73, 74, 82, 83, 84, 85, 86, 87, 88, 89}
Pattern 0 in week 4 can play against patterns {2, 3, 4, 5, 6, 7, 15, 16, 17, 18, 19, 20, 28, 2

In [4]:

from gurobipy import Model, GRB, quicksum

# Initialize model
m = Model("Sports_Scheduling")

# Parameters
patterns = range(90)  # Pattern indices (0 to 89)
weeks = range(9)      # Week indices (0 to 8)

# Binary decision variable: x[i, j, t] is 1 if pattern i is paired with pattern j in week t
x = m.addVars(patterns, patterns, weeks, vtype=GRB.BINARY, name="x")

# New binary decision variable y[i]: 1 if pattern i is selected as part of the 10 patterns, 0 otherwise
y = m.addVars(patterns, vtype=GRB.BINARY, name="y")

# Constraint: Select exactly 10 patterns to be used consistently across all weeks
m.addConstr(quicksum(y[i] for i in patterns) == 10, name="select_10_patterns")

# Constraint: Ensure each pattern used in a week is one of the 10 selected patterns
for i in patterns:
    for t in weeks:
        if (i,t) in feasible_opponents:
            m.addConstr(quicksum(x[i, j, t] for j in feasible_opponents[i,t]) == y[i], name=f"pattern_consistency_{i}_{t}")

# Constraint B2: Pairs of assigned patterns are equivalent (i vs j is the same as j vs i)
for i in patterns:
    for j in patterns:
            for t in weeks:
                m.addConstr(x[i, j, t] == x[j, i, t], name=f"B2_symmetry_{i}_{j}_{t}")


# Constraint B3: Each pair of selected patterns is selected for exactly one week
for i in patterns:
    for j in patterns:
            if i != j:
                m.addConstr(quicksum(x[i, j, t] for t in weeks) <= y[i], name=f"B3_unique_match_{i}_{j}")
                m.addConstr(quicksum(x[i, j, t] for t in weeks) <= y[j], name=f"B3_unique_match_{i}_{j}")





# Solve the model
m.optimize()

# Check and output solution
if m.status == GRB.OPTIMAL:
    print("Feasible schedule found")
    selected_patterns = [i for i in patterns if y[i].x > 0.5]
    schedule = {(i, j, t): x[i, j, t].x for i in patterns for j in patterns for t in weeks if x[i, j, t].x > 0.5 and i < j}
    print("Selected patterns:", selected_patterns)
    print(schedule)
else:
    print("No feasible solution found")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 89731 rows, 72990 columns and 341730 nonzeros
Model fingerprint: 0x7d3820f2
Variable types: 0 continuous, 72990 integer (72990 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 81162 rows and 54675 columns
Presolve time: 0.39s
Presolved: 8569 rows, 18315 columns, 81306 nonzeros
Variable types: 0 continuous, 18315 integer (18315 binary)

Root relaxation: objective 0.000000e+00, 4162 iterations, 0.91 seconds (1.42 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.

In [5]:
import pandas as pd

# Extract weeks dynamically
weeks = sorted({t for _, _, t in schedule.keys()})
calendar_data = {f"Week {t+1}": [] for t in weeks}

# Populate calendar data for each week
for t in weeks:
    week_pairs = []
    for (i, j, w) in schedule:
        if w == t:
            week_pairs.append(f"Pattern {i} vs Pattern {j}")
    calendar_data[f"Week {t+1}"] = week_pairs

# Create and display the DataFrame
calendar_df = pd.DataFrame(calendar_data)
calendar_df


,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9
0,Pattern 5 vs Pattern 61,Pattern 5 vs Pattern 75,Pattern 5 vs Pattern 18,Pattern 5 vs Pattern 71,Pattern 5 vs Pattern 12,Pattern 5 vs Pattern 83,Pattern 5 vs Pattern 30,Pattern 5 vs Pattern 66,Pattern 5 vs Pattern 24
1,Pattern 12 vs Pattern 71,Pattern 12 vs Pattern 24,Pattern 12 vs Pattern 83,Pattern 12 vs Pattern 61,Pattern 18 vs Pattern 83,Pattern 12 vs Pattern 75,Pattern 12 vs Pattern 66,Pattern 12 vs Pattern 18,Pattern 12 vs Pattern 30
2,Pattern 18 vs Pattern 75,Pattern 18 vs Pattern 71,Pattern 24 vs Pattern 61,Pattern 18 vs Pattern 24,Pattern 24 vs Pattern 75,Pattern 18 vs Pattern 30,Pattern 18 vs Pattern 61,Pattern 24 vs Pattern 30,Pattern 18 vs Pattern 66
3,Pattern 24 vs Pattern 66,Pattern 30 vs Pattern 66,Pattern 30 vs Pattern 71,Pattern 30 vs Pattern 75,Pattern 30 vs Pattern 61,Pattern 24 vs Pattern 71,Pattern 24 vs Pattern 83,Pattern 61 vs Pattern 75,Pattern 61 vs Pattern 71
4,Pattern 30 vs Pattern 83,Pattern 61 vs Pattern 83,Pattern 66 vs Pattern 75,Pattern 66 vs Pattern 83,Pattern 66 vs Pattern 71,Pattern 61 vs Pattern 66,Pattern 71 vs Pattern 75,Pattern 71 vs Pattern 83,Pattern 75 vs Pattern 83


## Step 3: Pattern to team allocation


In [6]:
# Create the list of teams and ranking
teams = [
    "Manchester City", "Liverpool", "Arsenal", 
    "Manchester United", "Chelsea", "Tottenham Hotspur", 
    "Newcastle United", "Brighton", "Aston Villa", "West Ham"
]

ranking = {
    1: teams[:3],  # Position 1
    2: teams[3:6],  # Position 2
    3: teams[6:]    # Position 3
}

# Assign a unique number to each team
team_numbers = {i : team for i, team in enumerate(teams)}

# Create a reverse mapping to look up the number of a team
team_to_number = {team: number for number, team in team_numbers.items()}

# Create a dictionary to store subsets of stronger teams
stronger_teams = {}

# Iterate through each team and determine stronger teams
for position, teams_at_position in ranking.items():
    for team in teams_at_position:
        team_number = team_to_number[team]  # Get the team's unique number
        stronger_teams[team_number] = [
            team_to_number[stronger_team]
            for higher_position in range(1, position)  # Look at higher positions only
            for stronger_team in ranking[higher_position]
        ]

# Print the team-to-number mapping and the stronger teams dictionary
print("Team Numbers:", team_numbers)
print("Stronger Teams:", stronger_teams)

Team Numbers: {0: 'Manchester City', 1: 'Liverpool', 2: 'Arsenal', 3: 'Manchester United', 4: 'Chelsea', 5: 'Tottenham Hotspur', 6: 'Newcastle United', 7: 'Brighton', 8: 'Aston Villa', 9: 'West Ham'}
Stronger Teams: {0: [], 1: [], 2: [], 3: [0, 1, 2], 4: [0, 1, 2], 5: [0, 1, 2], 6: [0, 1, 2, 3, 4, 5], 7: [0, 1, 2, 3, 4, 5], 8: [0, 1, 2, 3, 4, 5], 9: [0, 1, 2, 3, 4, 5]}


In [7]:
match_dict = schedule

# Initialize the dictionary to store matches for each team
pattern_matches = {}

# Process the match dictionary
for (team1, team2, week), value in match_dict.items():
    if value > 0.5:  # Only consider valid matches
        # Add team2 and week to team1's record
        if team1 not in pattern_matches:
            pattern_matches[team1] = set()
        pattern_matches[team1].add((team2, week))

        # Add team1 and week to team2's record
        if team2 not in pattern_matches:
            pattern_matches[team2] = set()
        pattern_matches[team2].add((team1, week))

# Display the resulting dictionary
pattern_matches

{5: {(12, 4),
  (18, 2),
  (24, 8),
  (30, 6),
  (61, 0),
  (66, 7),
  (71, 3),
  (75, 1),
  (83, 5)},
 12: {(5, 4),
  (18, 7),
  (24, 1),
  (30, 8),
  (61, 3),
  (66, 6),
  (71, 0),
  (75, 5),
  (83, 2)},
 18: {(5, 2),
  (12, 7),
  (24, 3),
  (30, 5),
  (61, 6),
  (66, 8),
  (71, 1),
  (75, 0),
  (83, 4)},
 24: {(5, 8),
  (12, 1),
  (18, 3),
  (30, 7),
  (61, 2),
  (66, 0),
  (71, 5),
  (75, 4),
  (83, 6)},
 30: {(5, 6),
  (12, 8),
  (18, 5),
  (24, 7),
  (61, 4),
  (66, 1),
  (71, 2),
  (75, 3),
  (83, 0)},
 61: {(5, 0),
  (12, 3),
  (18, 6),
  (24, 2),
  (30, 4),
  (66, 5),
  (71, 8),
  (75, 7),
  (83, 1)},
 66: {(5, 7),
  (12, 6),
  (18, 8),
  (24, 0),
  (30, 1),
  (61, 5),
  (71, 4),
  (75, 2),
  (83, 3)},
 71: {(5, 3),
  (12, 0),
  (18, 1),
  (24, 5),
  (30, 2),
  (61, 8),
  (66, 4),
  (75, 6),
  (83, 7)},
 75: {(5, 1),
  (12, 5),
  (18, 0),
  (24, 4),
  (30, 3),
  (61, 7),
  (66, 2),
  (71, 6),
  (83, 8)},
 83: {(5, 5),
  (12, 2),
  (18, 4),
  (24, 6),
  (30, 0),
  (61, 1),
  (6

In [ ]:
from gurobipy import Model, GRB, quicksum

# Initialize the model
m = Model("Team_to_Pattern_Allocation")

# Parameters
teams = range(10)  # Assuming 10 teams
patterns = selected_patterns # Unique patterns from the calendar
weeks = range(9)  # Week indices


# Given calendar: dictionary in the form {(i, j, t): 1.0}
calendar = schedule

# Decision variables
z = m.addVars(teams, patterns, vtype=GRB.BINARY, name="z")  # Team-to-pattern allocation
# s = m.addVars(teams, range(8), vtype=GRB.BINARY, name="s")  # Carry-over effect
x = m.addVars(teams, teams, weeks, vtype=GRB.BINARY, name="x")  # Matches between teams

# # Objective: Minimize carry-over effect
# m.setObjective(quicksum(s[i, k] for i in teams for k in range(8)), GRB.MINIMIZE)

# Constraint C2: Each team must be assigned to exactly one pattern
for i in teams:
    m.addConstr(quicksum(z[i, p] for p in patterns) == 1, name=f"team_pattern_allocation_{i}")

# Each pattern must be assigned to exactly one team
for p in patterns:
    m.addConstr(quicksum(z[i,p] for i in teams) == 1, name=f"team_pattern_unique_allocation_{p}")

# Constraint: Relate calendar pattern pairings to matches between teams

# for (p_i, p_j, t) in calendar.keys():
#     for i in teams:
#         for j in teams:
#             if i != j:
#                 m.addConstr(x[i, j, t] <= z[i, p_i], name=f"match_constraint_pattern_i_{i}_{j}_{t}")
#                 m.addConstr(x[i, j, t] <= z[j, p_j] , name=f"match_constraint_pattern_j_{i}_{j}_{t}")
#                 m.addConstr(x[i, j, t] >= z[i, p_i] + z[j, p_j] - 1, name=f"match_constraint_link_{i}_{j}_{t}")


for p_i in selected_patterns:
    for (p_j, t) in pattern_matches[p_i]:
        for i in teams:
            for j in teams:              
                m.addConstr(x[i, j, t] <= z[i, p_i], name=f"match_constraint_pattern_i_{i}_{j}_{t}")
                m.addConstr(x[i, j, t] <= z[j, p_j], name=f"match_constraint_pattern_j_{i}_{j}_{t}")
                m.addConstr(x[i, j, t] >= z[i, p_i] + z[j, p_j] - 1, name=f"match_constraint_link_{i}_{j}_{t}")



# # Constraint: Pairs of assigned teams are equivalent (i vs j is the same as j vs i)
# for i in teams:
#     for j in teams:
#         if j > i:
#             for t in weeks:
#                 m.addConstr(x[i, j, t] == x[j, i, t], name=f"match_symmetry_{i}_{j}_{t}")

# # Constraint C3: Carry-over effect between weeks
# for i in teams:
#     for k in range(8):  # 8 weeks for carry-over
#         m.addConstr(
#             quicksum(x[i, j, k] for j in stronger_teams[i]) + quicksum(x[i, j, k+1] for j in stronger_teams[i]) <= 1 + s[i, k],
#             name=f"carry_over_effect_{i}_{k}"
        # )

# Solve the model
m.optimize()

if m.status == GRB.OPTIMAL:
    print("Optimal solution found")
    allocation = {(i, p): z[i, p].x for i in teams for p in patterns if z[i, p].x > 0.5}
    # carry_over = {(i, k): s[i, k].x for i in teams for k in range(8) if s[i, k].x > 0.5}
    matches = {(i, j, t): x[i, j, t].x for i in teams for j in teams for t in weeks if x[i, j, t].x > 0.5}
    print("Team-to-pattern allocation:", allocation)
    # print("Carry-over effect:", carry_over)
    print("Matches between teams:", matches)
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12170 rows, 1000 columns and 36650 nonzeros
Model fingerprint: 0xef60935b
Variable types: 0 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 127 columns
Presolve time: 0.07s

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.14 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found


: 

In [16]:
import pandas as pd



# Extract actual team-to-pattern mapping
pattern_to_team = {p: team_numbers[team] for (team, p), value in allocation.items() if value > 0.5}

# Mockup of `calendar_df`
team_calendar = calendar_df

# Update patterns to actual team names in the DataFrame
def replace_patterns(match, mapping):
    team_1, team_2 = match.split(" vs ")
    team_1_name = mapping.get(int(team_1.split(" ")[1]), team_1)
    team_2_name = mapping.get(int(team_2.split(" ")[1]), team_2)
    return f"{team_1_name} vs {team_2_name}"

for col in team_calendar.columns:
    team_calendar[col] = team_calendar[col].apply(lambda x: replace_patterns(x, pattern_to_team))

# Display the updated DataFrame
print(team_calendar)


                           Week 1                                Week 2  \
0    Chelsea vs Tottenham Hotspur                   Chelsea vs West Ham   
1    Arsenal vs Manchester United                   Arsenal vs Brighton   
2           Liverpool vs West Ham        Liverpool vs Manchester United   
3    Brighton vs Newcastle United       Aston Villa vs Newcastle United   
4  Aston Villa vs Manchester City  Tottenham Hotspur vs Manchester City   

                             Week 3                               Week 4  \
0              Chelsea vs Liverpool         Chelsea vs Manchester United   
1        Arsenal vs Manchester City         Arsenal vs Tottenham Hotspur   
2     Brighton vs Tottenham Hotspur                Liverpool vs Brighton   
3  Aston Villa vs Manchester United              Aston Villa vs West Ham   
4      Newcastle United vs West Ham  Newcastle United vs Manchester City   

                                  Week 5  \
0                     Chelsea vs Arsenal   
1   